#### **This notebook concats all cosine similarity**

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import sys
import os

import importlib

#### packages
import helper.strategy_helper as st
import helper.visualization as viz_hp
import helper.helper as hp
import helper.file_helper as file_hp
import config.config as config_hp
import helper.pandas_helper as pd_hp

#### **Concat all cosine data**

In [ ]:
path = '/N/slate/potem/data/derived/reply_embedding/'
filename = 'replier_tweetid_combination*'
files = os.path.join(path, filename)
df_all = pd.DataFrame()
for file in glob.glob(files):
    print(file)
    
    if file == '/N/slate/potem/data/derived/reply_embedding/replier_tweetid_combination.pkl.gz':
        continue
        
    df = pd.read_pickle(file)
    
    df_all = df_all.append(df, ignore_index=True)

In [10]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458180200 entries, 0 to 1458180199
Data columns (total 4 columns):
 #   Column          Dtype  
---  ------          -----  
 0   poster_tweetid  object 
 1   replier_x       object 
 2   replier_y       object 
 3   cosine          float32
dtypes: float32(1), object(3)
memory usage: 38.0+ GB


In [12]:
config = config_hp.config()
balanced = config['BALANCED']

balanced_neg_conversation = balanced['balanced_neg_conversation']
balanced_pos_conversation = balanced['balanced_pos_conversation']

df_neg = pd.read_pickle(balanced_neg_conversation)
df_pos = pd.read_pickle(balanced_pos_conversation)

df_conv = df_neg.append(df_pos)

/tmp/ipykernel_2168346/416250505.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_conv = df_neg.append(df_pos)


In [13]:
df_conv.columns

Index(['tweet_text', 'conversation_id', 'replier_tweetid', 'replier_userid',
       'poster_userid', 'poster_tweetid', 'tweet_time', 'tweet_language',
       'replier_label', 'year', 'campaign', 'tweet_label', 'tweet_time_year',
       'common', 'id', 'username'],
      dtype='object')

In [17]:
grouped = df_conv.groupby(['poster_tweetid', 'tweet_label'])

# Get the keys of the groupby operation
keys = list(grouped.groups.keys())

# Create a DataFrame from the keys
df_grp = pd.DataFrame(data=keys, columns=['poster_tweetid','tweet_label'])

print(df_grp.head())

        poster_tweetid  tweet_label
0  1000004851579981824            1
1  1000082706007543808            1
2  1000082709685948416            1
3  1000086025161625600            1
4  1000096002135781376            1


In [ ]:
config = config_hp.config()
embedding_path = config['EMBEDDINGS_PATH']

cosine_pair_wise = embedding_path['cosine_pair_wise']

df_temp = df_all.merge(df_grp[['poster_tweetid',
                                'tweet_label'
                               ]])

df_temp.to_pickle(cosine_pair_wise)